In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

In [2]:
#Import required sklearn functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from collections import defaultdict

In [3]:
#Import sklearn classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [4]:
#Import library to oversample 
from imblearn.over_sampling import RandomOverSampler

In [5]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [6]:
#Allows figures to be visualized in jupyter notebook
%matplotlib inline

In [7]:
#Functions used in the study

#Remove those numbers from analysis data
def filter_rows_by_values1(df, col, values):
    return df[~df[col].isin(values)]

#Remove those numbers from analysis data
def filter_rows_by_values2(df, col, values):
    return df[df[col].isin(values)]

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

def remove_95correlated(correlated):
    #Remove any features that are greater than 95% correlated
    corr_matrix = correlated.corr()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

    correlated = correlated.drop(to_drop, axis = 1)
    corr_matrix = correlated.corr()
    return correlated

def remove_nonimportant(X_values, y_values):
    # Specifys Random Forest and the Number of Trees, SelectFromModel will
    # select features which are most important
    feature_names = [f"feature {i}" for i in range(X_values.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(X_values, y_values)

    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time

    threshold = np.sort(importances)[-100]
    
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(X_values, y_values)

    # Select the final features set 
    sel.get_support()
    selected_feat= X_values.columns[(sel.get_support())]

    # Prints the names of the final selected features
    print(selected_feat)
    X_values = X_values[selected_feat]
    
    return X_values

def dendrogram(X_values, y):
    corr = spearmanr(X_values).correlation
    # Ensure the correlation matrix is symmetric
    corr = (corr + corr.T) / 2
    np.fill_diagonal(corr, 1)
    distance_matrix = 1 - np.abs(corr)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
  
    trained_cluster_ids = hierarchy.fcluster(dist_linkage, y, criterion="distance")
    trained_cluster_id_to_feature_ids = defaultdict(list) 
    for idx, trained_cluster_id in enumerate(trained_cluster_ids):
        trained_cluster_id_to_feature_ids[trained_cluster_id].append(idx)
    
    trained_selected_features = [v[0] for v in trained_cluster_id_to_feature_ids.values()]
    final_selected_features = X_values.columns[trained_selected_features]
    X_train = X_values[final_selected_features]
    return X_train

def classificationMetrics(results, y_test, pred):
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred, average=None, zero_division=0)
    recall = recall_score(y_test, pred, average=None)
    F1 = f1_score(y_test, pred, average=None)
    comb = np.concatenate((x, y, acc, prec, recall, F1), axis=None)
    comb = [comb]
    results = results.append(pd.DataFrame(comb, columns=results.columns), ignore_index=True)
    return results


In [8]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)

In [9]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [10]:
# Read Training/Test data input File
data = pd.read_csv('BorylationTrainingTest 9-26-24.csv')
data.head()

,Substrate,Product,Boronic Ester,Active Catalyst-Ligand,Catalyst,Ligand,Buried_Vol,PyramidalizationAR,PyramidalizationG,SASA_area,SASA_vol,Sterimol_L,SterimolB_1,Sterimol_B_5,Buried_Sterimol_L,Buried_SterimolB_1,Buried_Sterimol_B_5,Hirshfeld Heavy Atom Charge,CM5 Charge,Hirshfeld Carbon Charge,Hirshfeld Hydrogen Charge,ESP Heavy Atom Charge,ESP Carbon Charge,ESP Hydrogen Charge,NPA Carbon Charge,NPA Hydrogen Charge,MBS Heavy Atom Charge,MBS Carbon Charge,MBS Hydrogen Charge,Mulliken Heavy Charge,Mulliken Carbon Charge,Mulliken Hydrogen Charge,Solvent,Temp,Aliphatic Atom Count,Aliphatic Bond Count,Aliphatic Ring Count,Aromatic Atom Count,Aromatic Bond Count,Aromatic Ring Count,Steric Effect Index,Atomic_Polarizability,Balaban Index,Chain Atom Count,Distance Degree,Dreiding Energy,Eccentricity,Harary Index,Hydrogen Acceptor Count,Hydrogen Acceptor Site Count,Heteroatom Aromatic Ring Count,Hydrogen Donor Count,Hydrogen Donor Site Count,Hyper Wiener Index,Largest Ring Size,Max Projection Area,Max Projection Radius,MaxZ,Moleculare Polarizability,Platt Index,Refractivity,Ring Atom Count,Rot Bond Count,Sigma Electronegativity,Wiener Index,Product_Ratio
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-6-7-8-9-10-11-12-15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.790830,7.558608,5.941608,4.790830,7.278100,-0.003389,-0.000578,-0.101759,0.032223,-0.078389,-0.182602,0.032537,-0.57256,0.19272,-0.008908,-0.254190,0.080765,-0.220320,-0.621212,0.132089,2,150,8,7,0,0,0,0,1.193207,1.116291,2.530060,8,28,12.13,7,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.522910,12,38.6102,0,5,7.387931,84,1
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-8-9-10-11-12(2)15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.790830,7.558608,5.941608,4.790830,7.278100,0.003094,0.000249,-0.057714,0.030404,0.098246,0.182934,-0.042344,-0.38255,0.18757,0.009321,-0.138428,0.073874,0.044556,-0.224805,0.134680,2,150,8,7,0,0,0,0,2.190292,1.116291,2.530060,8,22,12.13,6,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.522910,12,38.6102,0,5,7.470532,84,0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-9-10-11-12(8-2)15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.790830,7.558608,5.941608,4.790830,7.278100,-0.000052,-0.000032,-0.059257,0.029602,-0.011526,0.182934,-0.012016,-0.37899,0.18768,-0.000222,-0.141245,0.070512,0.012779,-0.261495,0.137137,2,150,8,7,0,0,0,0,2.310662,1.116291,2.530060,8,18,12.13,5,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.522910,12,38.6102,0,5,7.493894,84,0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-9-11-12(10-8-2)15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.790830,7.558608,5.941608,4.790830,7.278100,0.000299,0.000312,-0.058954,0.029626,-0.008331,0.037009,-0.022670,-0.37717,0.18811,-0.000191,-0.141973,0.070891,0.162986,-0.115385,0.139186,2,150,8,7,0,0,0,0,2.339699,1.116291,2.530060,8,16,12.13,4,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.522910,12,38.6102,0,5,7.496149,84,0
4,"InChI=1S/C7H15N/c1-2-8-6-4-3-5-7-8/h2-7H2,1H3","InChI=1S/C13H26BNO2/c1-12(2)13(3,4)17-14(16-12...",2,7,3,1,41.0,1.059976,9.242505,814.372698,1707.085805,6.090976,5.055852,9.142000,6.090976,5.055852,7.624077,0.002028,0.013777,-0.102309,0.036494,-0.043709,-0.156454,0.023226,-0.58186,0.20119,0.006751,-0.255789,0.085570,-0.086611,-0.497717,0.137539,3,150,8,8,1,0,0,0,2.188081,1.116291,2.125016,2,16,21.20,4,15.783333,1,1,0,0,0,122,6,43.497048,4.404837,6.112425,14.445701,18,36.8787,6,1,7.937482,64,1


In [11]:
#group the compounds by numbers
data['grouped'] = data.groupby('Substrate', sort=False).ngroup()
data[['Substrate','grouped']]

,Substrate,grouped
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
4,"InChI=1S/C7H15N/c1-2-8-6-4-3-5-7-8/h2-7H2,1H3",1
...,...,...
1022,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,198
1023,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,198
1024,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,198
1025,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,198


In [12]:
#Convert substrates to Mordred features
data = get_Mordred(data)

100%|██████████| 1027/1027 [01:31<00:00, 11.27it/s]


In [13]:
DTResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

NBResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

DTmaxacc_comb = pd.DataFrame()
NBmaxacc_comb = pd.DataFrame()

model_columns = pd.DataFrame()
for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 198,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(data, "grouped", set_numbers)
    training_data = training_data.drop('grouped', axis = 1)    
    test_data = filter_rows_by_values2(data, "grouped", set_numbers)
    test_data = test_data.drop('grouped', axis = 1)     
     
    #Remove features that dont change
    training_data = remove_zero_varience(training_data)
    
    #Remove features that are more than 95% correlated
    training_data = remove_95correlated(training_data)
    
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']
    
    #Remove features that are considered less important
    feature_names = [f"feature {i}" for i in range(training_X.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(training_X, training_y)
    
    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    
    threshold = np.sort(importances)[-100] 
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(training_X, training_y)
     
    # Select the reduced features set 
    sel.get_support()
    selected_feat= training_X.columns[(sel.get_support())]
    
    reduced1_X = training_X[selected_feat]
    test_X = test_X[selected_feat]
    
    #Apply over-sampling to dataset
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(reduced1_X, training_y) 
    
    for y in [ 
              0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35,
              0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 
              0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
              0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50
             ]:
    
        #Make final training and test set and save them as df's  
        X_train = dendrogram(X_resampled, y)
        test_X = test_X[X_train.columns]
        training_columns_list = X_train.columns.tolist()
        training_columns_list = (x, y, training_columns_list)
        training_columns_list = (pd.DataFrame(training_columns_list).T)
                                        
        #Decision Tree CLassifier
        DT = DecisionTreeClassifier(max_depth=9)
        DT.fit(X_train, y_resampled)
        pred_DT = DT.predict(test_X)
        DTResults_df = classificationMetrics(DTResults_df, test_y, pred_DT)
 
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = DTResults_df.groupby('y')['acc'].mean()
        acc_std = DTResults_df.groupby('y')['acc'].std()
        precision_0_mean = DTResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = DTResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = DTResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = DTResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = DTResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = DTResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = DTResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = DTResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = DTResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = DTResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = DTResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = DTResults_df.groupby('y')['F1 1'].std()
        
        DTaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        DTaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 
          
        
                
        DTmaxacc = DTaverage_df[DTaverage_df.acc_mean == DTaverage_df.acc_mean.max()]
        DTmaxacc_copy  = DTmaxacc.copy()
        DTmaxacc_copy['x_col'] = x       
        
        #Naive Bayes CLassifier
        NB = GaussianNB()
        NB.fit(X_train, y_resampled)
        pred_NB = NB.predict(test_X)
        NBResults_df = classificationMetrics(NBResults_df, test_y, pred_NB)
 
            #Determine the mean accuracy of the different dendrogram settings
        acc_mean = NBResults_df.groupby('y')['acc'].mean()
        acc_std = NBResults_df.groupby('y')['acc'].std()
        precision_0_mean = NBResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = NBResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = NBResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = NBResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = NBResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = NBResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = NBResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = NBResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = NBResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = NBResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = NBResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = NBResults_df.groupby('y')['F1 1'].std()
        
        NBaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        NBaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 

        NBmaxacc = NBaverage_df[NBaverage_df.acc_mean == NBaverage_df.acc_mean.max()]
        NBmaxacc_copy  = NBmaxacc.copy()
        NBmaxacc_copy['x_col'] = x

    DTmaxacc_comb = DTmaxacc_comb.append(DTmaxacc_copy)
    NBmaxacc_comb = NBmaxacc_comb.append(NBmaxacc_copy)

#Write the results onto a CSV file, currently commented out 
DTmaxacc_comb.to_csv("10Runs_ClassificationScreening_DT_NB.csv",)
NBmaxacc_comb.to_csv("10Runs_ClassificationScreening_DT_NB.csv", mode="a")


In [14]:
DTmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.50,0.740526,0.008613,0.895041,0.016305,0.415401,0.013839,0.763576,0.019257,0.651282,0.066425,0.823791,0.007192,0.506437,0.027240,1
0.35,0.701760,0.024555,0.854753,0.024200,0.330384,0.038274,0.756479,0.023686,0.484615,0.085609,0.802281,0.017064,0.391771,0.052225,2
0.35,0.725311,0.039717,0.869697,0.029236,0.365435,0.060027,0.773832,0.032006,0.529915,0.095782,0.818725,0.027703,0.431760,0.072099,3
0.35,0.725010,0.036151,0.879801,0.030893,0.360913,0.053808,0.765104,0.034942,0.560897,0.099424,0.817873,0.025653,0.437060,0.063641,4
0.45,0.721106,0.034520,0.878562,0.030475,0.356816,0.050638,0.761081,0.039880,0.558462,0.107706,0.814740,0.025394,0.432491,0.062823,5
0.45,0.731390,0.039318,0.877375,0.028274,0.375275,0.062949,0.776500,0.050529,0.549573,0.102386,0.822671,0.029372,0.441225,0.062105,6
0.45,0.729173,0.036853,0.878168,0.026267,0.370839,0.059350,0.772672,0.047712,0.552381,0.095111,0.820985,0.027533,0.439479,0.057769,7
0.45,0.726958,0.035140,0.875555,0.025604,0.368507,0.056096,0.771758,0.044856,0.545833,0.090970,0.819445,0.026204,0.436186,0.055044,8
0.35,0.732403,0.047440,0.880786,0.025792,0.378787,0.064953,0.773235,0.057335,0.564103,0.091849,0.822323,0.036940,0.449820,0.064983,9


In [15]:
NBmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.35,0.578947,0.000000,0.873684,0.000000,0.284211,0.000000,0.549669,0.000000,0.692308,0.000000,0.674797,0.000000,0.402985,0.000000,1
0.45,0.575399,0.001760,0.903556,0.039985,0.287421,0.011106,0.527921,0.022313,0.769231,0.105229,0.665350,0.006890,0.417711,0.027428,2
0.35,0.587259,0.029025,0.896354,0.024316,0.286943,0.003736,0.551059,0.053241,0.735043,0.080617,0.680364,0.036322,0.411681,0.010249,3
0.35,0.612319,0.050584,0.891219,0.022820,0.285630,0.003960,0.594375,0.088775,0.679487,0.119705,0.708498,0.058450,0.399669,0.022847,4
0.35,0.605855,0.046980,0.889213,0.020759,0.283049,0.006298,0.586059,0.080963,0.682051,0.106920,0.702676,0.053456,0.397996,0.020662,5
0.35,0.613386,0.046059,0.884067,0.022194,0.286800,0.010222,0.600583,0.080725,0.662393,0.107047,0.711327,0.052476,0.397730,0.018839,6
0.35,0.604057,0.048413,0.884428,0.020542,0.281865,0.015415,0.587482,0.081344,0.670330,0.100905,0.702082,0.053617,0.394245,0.019427,7
0.35,0.609279,0.047333,0.885113,0.019284,0.287294,0.020408,0.593295,0.077581,0.673077,0.094586,0.706879,0.051711,0.400292,0.024266,8
0.35,0.611287,0.044959,0.887342,0.019243,0.287780,0.019277,0.594811,0.073220,0.678063,0.090235,0.709075,0.049118,0.401906,0.023319,9
